# 02 - Governance Visualization Dashboard

This notebook provides advanced visualization tools for AI governance metrics. You'll learn to:

1. **Dashboard Creation** - Build multi-panel governance dashboards
2. **Risk Distribution** - Visualize risk score distributions across models
3. **Compliance Tracking** - Chart compliance status and trends
4. **Decision Audit** - Create audit trail visualizations
5. **Time Series Analysis** - Track approval rates over time
6. **Heatmaps** - Analyze access patterns across dimensions

## Prerequisites

Ensure OPA and the example policies are running:

```bash
# From project root
docker compose up opa -d

# Load AI model policies from Example 02
cd examples/02-ai-model-approval
docker compose up -d
```

---

## 1. Setup and Configuration

Configure the environment for Docker-compatible visualization.

In [ ]:
# Set headless backend BEFORE importing matplotlib (required for Docker)
import os  # noqa: I001

os.environ["MPLBACKEND"] = "Agg"

# Standard library
import random
from datetime import datetime, timedelta
from typing import Any

# Data manipulation and visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from requests.exceptions import RequestException

# Enable inline plots
%matplotlib inline

# Set visualization style
sns.set_theme(style="whitegrid", palette="husl")

# Color palette for governance visualizations
COLORS = {
    "approved": "#27AE60",  # Green
    "denied": "#E74C3C",    # Red
    "pending": "#F39C12",   # Orange
    "low_risk": "#3498DB",  # Blue
    "medium_risk": "#F39C12",  # Orange
    "high_risk": "#E74C3C",    # Red
}


## 2. OPA Connection and Helper Functions

Establish connection to OPA and create helper functions for policy evaluation.

In [ ]:
# OPA URL configuration
OPA_URL = os.getenv("OPA_URL", "http://localhost:8181")


In [ ]:
def check_opa_health() -> bool:
    """Check if OPA is running and healthy."""
    try:
        response = requests.get(f"{OPA_URL}/health", timeout=5)
        if response.status_code == 200:
            return True
        else:
            return False
    except RequestException:
        return False


def evaluate_policy(policy_path: str, input_data: dict[str, Any]) -> dict[str, Any]:
    """Evaluate an OPA policy with the given input data."""
    response = requests.post(
        f"{OPA_URL}/v1/data/{policy_path}",
        json={"input": input_data},
        timeout=10,
    )
    response.raise_for_status()
    return response.json()


def safe_evaluate(policy_path: str, input_data: dict[str, Any]) -> dict[str, Any] | None:
    """Evaluate a policy with error handling."""
    try:
        return evaluate_policy(policy_path, input_data)
    except RequestException:
        return None


# Verify OPA connection
opa_healthy = check_opa_health()

## 3. Generate Simulated Governance Data

For demonstration purposes, we'll create realistic governance data. In production, this data would come from your actual governance logs and OPA decision records.

In [ ]:
def generate_model_data(num_models: int = 100) -> pd.DataFrame:
    """Generate simulated AI model governance data."""
    np.random.seed(42)  # For reproducibility
    
    # Model names
    model_types = ["sentiment", "classification", "recommendation", "chatbot", "vision"]
    
    # Generate data
    data = {
        "model_id": [f"model-{i:03d}" for i in range(num_models)],
        "model_type": [random.choice(model_types) for _ in range(num_models)],
        "risk_score": np.random.beta(2, 5, num_models),  # Skewed towards lower risk
        "environment": np.random.choice(
            ["development", "staging", "production"],
            num_models,
            p=[0.4, 0.35, 0.25],
        ),
        "bias_tested": np.random.choice([True, False], num_models, p=[0.8, 0.2]),
        "documentation_complete": np.random.choice([True, False], num_models, p=[0.7, 0.3]),
        "security_reviewed": np.random.choice([True, False], num_models, p=[0.85, 0.15]),
        "has_reviewer": np.random.choice([True, False], num_models, p=[0.6, 0.4]),
    }
    
    df = pd.DataFrame(data)
    
    # Calculate risk category
    df["risk_category"] = pd.cut(
        df["risk_score"],
        bins=[0, 0.3, 0.7, 1.0],
        labels=["low", "medium", "high"],
    )
    
    # Calculate compliance score
    df["compliance_score"] = (
        df["bias_tested"].astype(int) +
        df["documentation_complete"].astype(int) +
        df["security_reviewed"].astype(int)
    ) / 3
    
    return df


# Generate data
model_df = generate_model_data(100)

model_df.head()

In [ ]:
def evaluate_models_with_opa(df: pd.DataFrame) -> pd.DataFrame:
    """Evaluate each model against OPA policies.
    
    If OPA is not available, simulates decisions based on the data.
    """
    decisions = []
    
    for _, row in df.iterrows():
        # Build OPA input
        opa_input = {
            "model": {
                "id": row["model_id"],
                "risk_score": float(row["risk_score"]),
                "environment": row["environment"],
            },
            "compliance": {
                "bias_tested": bool(row["bias_tested"]),
                "documentation_complete": bool(row["documentation_complete"]),
                "security_reviewed": bool(row["security_reviewed"]),
            },
            "deployment": {"environment": row["environment"]},
        }
        
        if row["has_reviewer"]:
            opa_input["reviewer"] = {"id": "reviewer-001", "role": "senior_engineer"}
        
        # Try OPA evaluation
        result = safe_evaluate("ai/model_approval/allow", opa_input)
        
        if result is not None:
            decisions.append(result.get("result", False))
        else:
            # Simulate decision logic if OPA unavailable
            risk = row["risk_score"]
            compliant = row["compliance_score"] >= 0.67  # 2/3 compliance
            env = row["environment"]
            
            if risk < 0.3:
                # Low risk: auto-approve
                approved = True
            elif risk < 0.7:
                # Medium risk: needs compliance or reviewer
                approved = compliant or row["has_reviewer"]
            else:
                # High risk: always needs reviewer
                approved = row["has_reviewer"] and (env != "production" or compliant)
            
            decisions.append(approved)
    
    df = df.copy()
    df["approved"] = decisions
    return df


# Evaluate models
model_df = evaluate_models_with_opa(model_df)

approval_rate = model_df["approved"].mean() * 100

## 4. Governance Dashboard Overview

Create a comprehensive multi-panel dashboard showing key governance metrics at a glance.

In [ ]:
def create_governance_dashboard(df: pd.DataFrame) -> None:
    """Create a comprehensive governance dashboard with multiple visualizations."""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle("AI Governance Dashboard", fontsize=16, fontweight="bold", y=1.02)
    
    # 1. Approval Status (Pie Chart)
    ax1 = axes[0, 0]
    approved = df["approved"].sum()
    denied = len(df) - approved
    ax1.pie(
        [approved, denied],
        labels=["Approved", "Denied"],
        colors=[COLORS["approved"], COLORS["denied"]],
        autopct="%1.1f%%",
        startangle=90,
        explode=[0.05, 0],
    )
    ax1.set_title("Approval Status", fontweight="bold")
    
    # 2. Risk Distribution (Histogram)
    ax2 = axes[0, 1]
    ax2.hist(
        [df[df["approved"]]["risk_score"], df[~df["approved"]]["risk_score"]],
        bins=20,
        stacked=True,
        color=[COLORS["approved"], COLORS["denied"]],
        label=["Approved", "Denied"],
        edgecolor="white",
    )
    ax2.set_xlabel("Risk Score")
    ax2.set_ylabel("Count")
    ax2.set_title("Risk Score Distribution", fontweight="bold")
    ax2.legend()
    ax2.axvline(x=0.3, color="gray", linestyle="--", alpha=0.7, label="Low/Medium")
    ax2.axvline(x=0.7, color="gray", linestyle="-.", alpha=0.7, label="Medium/High")
    
    # 3. Approval by Environment (Bar Chart)
    ax3 = axes[0, 2]
    env_data = df.groupby(["environment", "approved"]).size().unstack(fill_value=0)
    env_data.columns = ["Denied", "Approved"]
    env_data[["Approved", "Denied"]].plot(
        kind="bar",
        stacked=True,
        color=[COLORS["approved"], COLORS["denied"]],
        ax=ax3,
    )
    ax3.set_xlabel("Environment")
    ax3.set_ylabel("Count")
    ax3.set_title("Decisions by Environment", fontweight="bold")
    ax3.tick_params(axis="x", rotation=0)
    ax3.legend(title="Decision")
    
    # 4. Compliance Status (Heatmap)
    ax4 = axes[1, 0]
    compliance_bins = ["Low", "Medium", "Full"]
    compliance_matrix = pd.crosstab(
        df["risk_category"],
        pd.cut(
            df["compliance_score"],
            bins=[0, 0.33, 0.67, 1.0],
            labels=compliance_bins,
        ),
    )
    sns.heatmap(
        compliance_matrix,
        annot=True,
        fmt="d",
        cmap="YlOrRd",
        ax=ax4,
    )
    ax4.set_xlabel("Compliance Level")
    ax4.set_ylabel("Risk Category")
    ax4.set_title("Risk vs Compliance", fontweight="bold")
    
    # 5. Model Type Distribution (Horizontal Bar)
    ax5 = axes[1, 1]
    type_approval = df.groupby("model_type")["approved"].mean().sort_values()
    bar_colors = [
        COLORS["approved"] if rate > 0.5 else COLORS["denied"]
        for rate in type_approval
    ]
    type_approval.plot(
        kind="barh",
        color=bar_colors,
        ax=ax5,
    )
    ax5.set_xlabel("Approval Rate")
    ax5.set_ylabel("Model Type")
    ax5.set_title("Approval Rate by Model Type", fontweight="bold")
    ax5.set_xlim(0, 1)
    ax5.axvline(x=0.5, color="gray", linestyle="--", alpha=0.7)
    
    # 6. Key Metrics Summary (Text)
    ax6 = axes[1, 2]
    ax6.axis("off")
    
    total = len(df)
    approved_pct = df["approved"].mean() * 100
    avg_risk = df["risk_score"].mean()
    avg_compliance = df["compliance_score"].mean() * 100
    high_risk_denied = df[(df["risk_category"] == "high") & (~df["approved"])].shape[0]
    
    metrics_text = (
        f"KEY METRICS SUMMARY\n"
        f"{'='*30}\n\n"
        f"Total Models Evaluated: {total}\n\n"
        f"Approval Rate: {approved_pct:.1f}%\n\n"
        f"Average Risk Score: {avg_risk:.3f}\n\n"
        f"Average Compliance: {avg_compliance:.1f}%\n\n"
        f"High-Risk Denials: {high_risk_denied}\n\n"
        f"{'='*30}\n"
        f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}"
    )
    ax6.text(
        0.1, 0.5,
        metrics_text,
        fontsize=12,
        fontfamily="monospace",
        verticalalignment="center",
        bbox={"boxstyle": "round", "facecolor": "lightgray", "alpha": 0.8},
    )
    ax6.set_title("Summary", fontweight="bold")
    
    plt.tight_layout()
    plt.show()
    
    # Close figure to prevent memory leaks
    plt.close(fig)


# Create the dashboard
create_governance_dashboard(model_df)

## 5. Risk Distribution Analysis

Deep dive into risk score distributions with violin plots and box plots.

In [ ]:
def visualize_risk_distribution(df: pd.DataFrame) -> None:
    """Create detailed risk distribution visualizations."""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    fig.suptitle("Risk Score Analysis", fontsize=14, fontweight="bold")
    
    # 1. Violin Plot by Environment
    ax1 = axes[0]
    sns.violinplot(
        data=df,
        x="environment",
        y="risk_score",
        hue="approved",
        split=True,
        palette={True: COLORS["approved"], False: COLORS["denied"]},
        ax=ax1,
    )
    ax1.set_title("Risk by Environment")
    ax1.set_xlabel("Environment")
    ax1.set_ylabel("Risk Score")
    ax1.legend(title="Approved", labels=["No", "Yes"])
    
    # 2. Box Plot by Model Type
    ax2 = axes[1]
    order = df.groupby("model_type")["risk_score"].median().sort_values().index
    sns.boxplot(
        data=df,
        x="model_type",
        y="risk_score",
        order=order,
        palette="coolwarm",
        ax=ax2,
    )
    ax2.set_title("Risk by Model Type")
    ax2.set_xlabel("Model Type")
    ax2.set_ylabel("Risk Score")
    ax2.tick_params(axis="x", rotation=45)
    
    # Add risk threshold lines
    ax2.axhline(y=0.3, color="orange", linestyle="--", alpha=0.7, label="Low threshold")
    ax2.axhline(y=0.7, color="red", linestyle="--", alpha=0.7, label="High threshold")
    
    # 3. KDE Plot
    ax3 = axes[2]
    for category in ["low", "medium", "high"]:
        subset = df[df["risk_category"] == category]
        color = {
            "low": COLORS["low_risk"],
            "medium": COLORS["medium_risk"],
            "high": COLORS["high_risk"],
        }[category]
        sns.kdeplot(
            data=subset,
            x="risk_score",
            fill=True,
            alpha=0.3,
            color=color,
            label=f"{category.capitalize()} Risk",
            ax=ax3,
        )
    ax3.set_title("Risk Score Density")
    ax3.set_xlabel("Risk Score")
    ax3.set_ylabel("Density")
    ax3.legend()
    
    plt.tight_layout()
    plt.show()
    plt.close(fig)


visualize_risk_distribution(model_df)

## 6. Compliance Tracking

Visualize compliance requirements and their impact on approval decisions.

In [ ]:
def visualize_compliance(df: pd.DataFrame) -> None:
    """Visualize compliance requirements and their impact."""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    fig.suptitle("Compliance Analysis", fontsize=14, fontweight="bold")
    
    # 1. Compliance Requirements Breakdown
    ax1 = axes[0]
    requirements = {
        "Bias Tested": df["bias_tested"].sum(),
        "Documentation": df["documentation_complete"].sum(),
        "Security Review": df["security_reviewed"].sum(),
    }
    total = len(df)
    
    x = list(requirements.keys())
    compliant = list(requirements.values())
    non_compliant = [total - c for c in compliant]
    
    bar_width = 0.6
    ax1.bar(
        x, compliant,
        width=bar_width,
        color=COLORS["approved"],
        label="Compliant",
    )
    ax1.bar(
        x, non_compliant,
        bottom=compliant,
        width=bar_width,
        color=COLORS["denied"],
        label="Non-Compliant",
    )
    ax1.set_ylabel("Count")
    ax1.set_title("Compliance by Requirement")
    ax1.legend()
    ax1.tick_params(axis="x", rotation=15)
    
    # 2. Compliance Score vs Approval Rate
    ax2 = axes[1]
    bin_labels = ["0-25%", "25-50%", "50-75%", "75-100%"]
    compliance_bins = pd.cut(
        df["compliance_score"],
        bins=4,
        labels=bin_labels,
    )
    approval_by_compliance = (
        df.groupby(compliance_bins, observed=True)["approved"].mean() * 100
    )
    
    bar_colors = [
        COLORS["denied"],
        COLORS["medium_risk"],
        COLORS["pending"],
        COLORS["approved"],
    ]
    bars = ax2.bar(
        approval_by_compliance.index.astype(str),
        approval_by_compliance.values,
        color=bar_colors,
        edgecolor="white",
    )
    ax2.set_xlabel("Compliance Score Range")
    ax2.set_ylabel("Approval Rate (%)")
    ax2.set_title("Compliance Impact on Approval")
    ax2.set_ylim(0, 100)
    
    # Add value labels
    for bar, val in zip(bars, approval_by_compliance.values, strict=False):
        ax2.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 2,
            f"{val:.0f}%",
            ha="center",
            fontsize=10,
        )
    
    # 3. Reviewer Impact
    ax3 = axes[2]
    reviewer_impact = (
        df.groupby(["risk_category", "has_reviewer"], observed=True)["approved"]
        .mean() * 100
    )
    reviewer_impact = reviewer_impact.unstack()
    reviewer_impact.columns = ["No Reviewer", "Has Reviewer"]
    
    reviewer_impact.plot(
        kind="bar",
        color=[COLORS["denied"], COLORS["approved"]],
        ax=ax3,
    )
    ax3.set_xlabel("Risk Category")
    ax3.set_ylabel("Approval Rate (%)")
    ax3.set_title("Reviewer Impact by Risk Level")
    ax3.tick_params(axis="x", rotation=0)
    ax3.legend(title="Reviewer")
    ax3.set_ylim(0, 100)
    
    plt.tight_layout()
    plt.show()
    plt.close(fig)


visualize_compliance(model_df)

## 7. Decision Audit Trail

Create visualizations for tracking and auditing governance decisions over time.

In [ ]:
def generate_time_series_data(df: pd.DataFrame, days: int = 30) -> pd.DataFrame:
    """Generate time series governance data for the past N days."""
    np.random.seed(42)
    
    dates = []
    decisions = []
    risk_categories = []
    environments = []
    
    base_date = datetime.now()
    envs = ["development", "staging", "production"]
    env_probs = [0.4, 0.35, 0.25]
    
    # Daily decisions with some trend
    for day in range(days):
        date = base_date - timedelta(days=days - day - 1)
        
        # Number of evaluations per day (varies)
        num_evals = np.random.poisson(lam=15)  # Average 15 per day
        
        # Approval rate trend (improving over time)
        base_approval_rate = 0.65 + (day / days) * 0.15
        
        for _ in range(num_evals):
            dates.append(date)
            decisions.append(np.random.random() < base_approval_rate)
            risk_cat = np.random.choice(
                ["low", "medium", "high"],
                p=[0.5, 0.35, 0.15],
            )
            risk_categories.append(risk_cat)
            environments.append(np.random.choice(envs, p=env_probs))
    
    return pd.DataFrame({
        "date": dates,
        "approved": decisions,
        "risk_category": risk_categories,
        "environment": environments,
    })


# Generate time series data
ts_df = generate_time_series_data(model_df, days=30)

ts_df.head()

In [ ]:
def visualize_audit_trail(df: pd.DataFrame) -> None:
    """Create audit trail visualizations."""
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle("Governance Decision Audit Trail", fontsize=14, fontweight="bold")
    
    # Aggregate by date
    daily = df.groupby("date").agg(
        total=("approved", "count"),
        approved=("approved", "sum"),
    ).reset_index()
    daily["denied"] = daily["total"] - daily["approved"]
    daily["approval_rate"] = daily["approved"] / daily["total"] * 100
    
    # 1. Daily Decision Volume
    ax1 = axes[0, 0]
    ax1.bar(
        daily["date"],
        daily["approved"],
        color=COLORS["approved"],
        label="Approved",
        alpha=0.8,
    )
    ax1.bar(
        daily["date"],
        daily["denied"],
        bottom=daily["approved"],
        color=COLORS["denied"],
        label="Denied",
        alpha=0.8,
    )
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Number of Decisions")
    ax1.set_title("Daily Decision Volume")
    ax1.legend()
    ax1.tick_params(axis="x", rotation=45)
    
    # 2. Approval Rate Trend
    ax2 = axes[0, 1]
    ax2.plot(
        daily["date"],
        daily["approval_rate"],
        color=COLORS["low_risk"],
        linewidth=2,
        marker="o",
        markersize=4,
    )
    ax2.fill_between(
        daily["date"],
        daily["approval_rate"],
        alpha=0.3,
        color=COLORS["low_risk"],
    )
    ax2.axhline(y=80, color="green", linestyle="--", alpha=0.7, label="Target (80%)")
    ax2.set_xlabel("Date")
    ax2.set_ylabel("Approval Rate (%)")
    ax2.set_title("Approval Rate Trend")
    ax2.set_ylim(0, 100)
    ax2.legend()
    ax2.tick_params(axis="x", rotation=45)
    
    # 3. Decisions by Risk Category (Stacked Area)
    ax3 = axes[1, 0]
    risk_daily = df.groupby(["date", "risk_category"]).size().unstack(fill_value=0)
    risk_daily = risk_daily.reindex(columns=["low", "medium", "high"])
    
    ax3.stackplot(
        risk_daily.index,
        risk_daily["low"],
        risk_daily["medium"],
        risk_daily["high"],
        labels=["Low Risk", "Medium Risk", "High Risk"],
        colors=[COLORS["low_risk"], COLORS["medium_risk"], COLORS["high_risk"]],
        alpha=0.8,
    )
    ax3.set_xlabel("Date")
    ax3.set_ylabel("Number of Evaluations")
    ax3.set_title("Evaluations by Risk Category")
    ax3.legend(loc="upper left")
    ax3.tick_params(axis="x", rotation=45)
    
    # 4. Environment Distribution Over Time
    ax4 = axes[1, 1]
    env_weekly = df.copy()
    env_weekly["week"] = env_weekly["date"].dt.isocalendar().week
    env_pivot = env_weekly.groupby(["week", "environment"]).size().unstack(fill_value=0)
    env_pivot = env_pivot.reindex(columns=["development", "staging", "production"])
    
    env_pivot.plot(
        kind="bar",
        stacked=True,
        color=["#3498DB", "#F39C12", "#9B59B6"],
        ax=ax4,
    )
    ax4.set_xlabel("Week Number")
    ax4.set_ylabel("Number of Decisions")
    ax4.set_title("Decisions by Environment (Weekly)")
    ax4.legend(title="Environment")
    ax4.tick_params(axis="x", rotation=0)
    
    plt.tight_layout()
    plt.show()
    plt.close(fig)


visualize_audit_trail(ts_df)

## 8. Advanced Heatmaps

Create heatmaps to analyze multi-dimensional governance patterns.

In [ ]:
def create_governance_heatmaps(df: pd.DataFrame) -> None:
    """Create detailed heatmaps for governance analysis."""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle("Governance Pattern Heatmaps", fontsize=14, fontweight="bold")
    
    # 1. Model Type vs Environment Approval Rate
    ax1 = axes[0]
    pivot1 = pd.pivot_table(
        df,
        values="approved",
        index="model_type",
        columns="environment",
        aggfunc="mean",
    ) * 100
    
    sns.heatmap(
        pivot1,
        annot=True,
        fmt=".0f",
        cmap="RdYlGn",
        center=50,
        vmin=0,
        vmax=100,
        cbar_kws={"label": "Approval Rate (%)"},
        ax=ax1,
    )
    ax1.set_title("Type × Environment")
    ax1.set_xlabel("Environment")
    ax1.set_ylabel("Model Type")
    
    # 2. Risk Category vs Compliance Level
    ax2 = axes[1]
    df_temp = df.copy()
    df_temp["compliance_level"] = pd.cut(
        df_temp["compliance_score"],
        bins=[0, 0.33, 0.67, 1.0],
        labels=["Low", "Medium", "Full"],
    )
    
    pivot2 = pd.pivot_table(
        df_temp,
        values="approved",
        index="risk_category",
        columns="compliance_level",
        aggfunc="mean",
    ) * 100
    
    # Reorder index for logical ordering
    pivot2 = pivot2.reindex(["low", "medium", "high"])
    
    sns.heatmap(
        pivot2,
        annot=True,
        fmt=".0f",
        cmap="RdYlGn",
        center=50,
        vmin=0,
        vmax=100,
        cbar_kws={"label": "Approval Rate (%)"},
        ax=ax2,
    )
    ax2.set_title("Risk × Compliance")
    ax2.set_xlabel("Compliance Level")
    ax2.set_ylabel("Risk Category")
    
    # 3. Correlation Matrix of Factors
    ax3 = axes[2]
    numeric_cols = [
        "risk_score", "bias_tested", "documentation_complete",
        "security_reviewed", "has_reviewer", "approved",
    ]
    corr_df = df[numeric_cols].copy()
    for col in numeric_cols:
        corr_df[col] = corr_df[col].astype(float)
    
    correlation = corr_df.corr()
    
    mask = np.triu(np.ones_like(correlation, dtype=bool))
    sns.heatmap(
        correlation,
        mask=mask,
        annot=True,
        fmt=".2f",
        cmap="coolwarm",
        center=0,
        vmin=-1,
        vmax=1,
        cbar_kws={"label": "Correlation"},
        ax=ax3,
    )
    ax3.set_title("Factor Correlation")
    
    plt.tight_layout()
    plt.show()
    plt.close(fig)


create_governance_heatmaps(model_df)

## 9. Interactive Policy Testing with Visualization

Combine policy evaluation with immediate visualization feedback.

In [ ]:
def analyze_scenario_batch(
    scenarios: list[dict],
    policy_path: str = "hello/allow",
) -> pd.DataFrame:
    """Evaluate multiple scenarios and return results as DataFrame."""
    results = []
    
    for i, scenario in enumerate(scenarios):
        result = safe_evaluate(policy_path, scenario)
        
        if result is not None:
            approved = result.get("result", False)
        else:
            # Simulate if OPA unavailable
            approved = scenario.get("user", {}).get("role") in ["admin"]
        
        results.append({
            "scenario_id": i + 1,
            "role": scenario.get("user", {}).get("role", "unknown"),
            "action": scenario.get("action", "unknown"),
            "resource": scenario.get("resource", "unknown"),
            "approved": approved,
        })
    
    return pd.DataFrame(results)


def visualize_scenario_results(df: pd.DataFrame) -> None:
    """Visualize scenario batch results."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 1. Results Grid
    ax1 = axes[0]
    pivot = df.pivot_table(
        index="role",
        columns="action",
        values="approved",
        aggfunc="first",
    ).fillna(False).astype(int)
    
    sns.heatmap(
        pivot,
        annot=True,
        fmt="d",
        cmap="RdYlGn",
        cbar_kws={"label": "Approved (1) / Denied (0)"},
        ax=ax1,
    )
    ax1.set_title("Scenario Results: Role × Action")
    ax1.set_xlabel("Action")
    ax1.set_ylabel("Role")
    
    # 2. Summary Bar Chart
    ax2 = axes[1]
    role_summary = df.groupby("role")["approved"].agg(["sum", "count"])
    role_summary["denied"] = role_summary["count"] - role_summary["sum"]
    role_summary = role_summary.rename(columns={"sum": "approved"})
    
    role_summary[["approved", "denied"]].plot(
        kind="bar",
        stacked=True,
        color=[COLORS["approved"], COLORS["denied"]],
        ax=ax2,
    )
    ax2.set_title("Decisions by Role")
    ax2.set_xlabel("Role")
    ax2.set_ylabel("Count")
    ax2.tick_params(axis="x", rotation=0)
    ax2.legend(title="Decision")
    
    plt.tight_layout()
    plt.show()
    plt.close(fig)


# Define test scenarios
test_scenarios = [
    {"user": {"role": "admin"}, "action": "read", "resource": "policy"},
    {"user": {"role": "admin"}, "action": "write", "resource": "policy"},
    {"user": {"role": "admin"}, "action": "delete", "resource": "policy"},
    {"user": {"role": "developer"}, "action": "read", "resource": "policy"},
    {"user": {"role": "developer"}, "action": "write", "resource": "policy"},
    {"user": {"role": "developer"}, "action": "delete", "resource": "policy"},
    {"user": {"role": "viewer"}, "action": "read", "resource": "policy"},
    {"user": {"role": "viewer"}, "action": "write", "resource": "policy"},
    {"user": {"role": "guest"}, "action": "read", "resource": "policy"},
]

# Run scenarios and visualize
scenario_results = analyze_scenario_batch(test_scenarios)

visualize_scenario_results(scenario_results)

## 10. Custom Visualization Builder

Create your own visualizations using the patterns demonstrated above.

In [ ]:
# EXERCISE: Create a custom visualization
#
# Use the model_df DataFrame to create your own chart.
# Available columns:
#   - model_id, model_type, risk_score, risk_category
#   - environment, bias_tested, documentation_complete, security_reviewed
#   - has_reviewer, compliance_score, approved

# Example: Scatter plot of risk vs compliance
fig, ax = plt.subplots(figsize=(10, 6))

for approved_status, group in model_df.groupby("approved"):
    color = COLORS["approved"] if approved_status else COLORS["denied"]
    label = "Approved" if approved_status else "Denied"
    ax.scatter(
        group["risk_score"],
        group["compliance_score"],
        c=color,
        label=label,
        alpha=0.6,
        s=50,
    )

ax.set_xlabel("Risk Score")
ax.set_ylabel("Compliance Score")
ax.set_title("Risk vs Compliance (colored by approval)")
ax.legend()

# Add quadrant lines
ax.axhline(y=0.67, color="gray", linestyle="--", alpha=0.5)
ax.axvline(x=0.5, color="gray", linestyle="--", alpha=0.5)

# Add quadrant labels
ax.text(0.25, 0.85, "Low Risk\nHigh Compliance", ha="center", fontsize=9, alpha=0.7)
ax.text(0.75, 0.85, "High Risk\nHigh Compliance", ha="center", fontsize=9, alpha=0.7)
ax.text(0.25, 0.33, "Low Risk\nLow Compliance", ha="center", fontsize=9, alpha=0.7)
ax.text(0.75, 0.33, "High Risk\nLow Compliance", ha="center", fontsize=9, alpha=0.7)

plt.tight_layout()
plt.show()
plt.close(fig)

In [ ]:
# EXERCISE: Add your own analysis
#
# Try creating:
# 1. A pie chart of risk categories
# 2. A line chart of cumulative approvals
# 3. A box plot comparing compliance across environments

# Your code here:

fig, ax = plt.subplots(figsize=(8, 6))

# Example: Risk category pie chart
risk_counts = model_df["risk_category"].value_counts()
pie_colors = [COLORS["low_risk"], COLORS["medium_risk"], COLORS["high_risk"]]

ax.pie(
    risk_counts,
    labels=[f"{cat.capitalize()} Risk" for cat in risk_counts.index],
    colors=pie_colors,
    autopct="%1.1f%%",
    startangle=90,
)
ax.set_title("Distribution of Risk Categories")

plt.tight_layout()
plt.show()
plt.close(fig)

## 11. Export and Reporting

Save visualizations and generate governance reports.

In [ ]:
def generate_governance_report(df: pd.DataFrame) -> str:
    """Generate a text-based governance summary report."""
    total = len(df)
    approved = df["approved"].sum()
    denied = total - approved
    approval_rate = df["approved"].mean() * 100
    
    # Calculate risk counts
    low_risk_count = (df["risk_category"] == "low").sum()
    low_risk_pct = (df["risk_category"] == "low").mean() * 100
    med_risk_count = (df["risk_category"] == "medium").sum()
    med_risk_pct = (df["risk_category"] == "medium").mean() * 100
    high_risk_count = (df["risk_category"] == "high").sum()
    high_risk_pct = (df["risk_category"] == "high").mean() * 100
    
    # Compliance metrics
    bias_pct = df["bias_tested"].mean() * 100
    doc_pct = df["documentation_complete"].mean() * 100
    sec_pct = df["security_reviewed"].mean() * 100
    avg_compliance = df["compliance_score"].mean() * 100
    
    report = f"""
╔═══════════════════════════════════════════════════════╗
║           AI GOVERNANCE SUMMARY REPORT                ║
╠═══════════════════════════════════════════════════════╣
║  Report: {datetime.now().strftime('%Y-%m-%d %H:%M:%S'):>45} ║
╠═══════════════════════════════════════════════════════╣
║  OVERALL METRICS                                      ║
║  ─────────────────────────────────────────────────    ║
║  Total Models:    {total:>37} ║
║  Approved:        {approved:>37} ║
║  Denied:          {denied:>37} ║
║  Approval Rate:   {approval_rate:>36.1f}% ║
╠═══════════════════════════════════════════════════════╣
║  RISK DISTRIBUTION                                    ║
║  ─────────────────────────────────────────────────    ║
║  Low Risk:    {low_risk_count:>5} ({low_risk_pct:>5.1f}%)                        ║
║  Medium Risk: {med_risk_count:>5} ({med_risk_pct:>5.1f}%)                        ║
║  High Risk:   {high_risk_count:>5} ({high_risk_pct:>5.1f}%)                        ║
╠═══════════════════════════════════════════════════════╣
║  ENVIRONMENT BREAKDOWN                                ║
║  ─────────────────────────────────────────────────    ║"""
    
    for env in ["development", "staging", "production"]:
        env_df = df[df["environment"] == env]
        env_rate = env_df["approved"].mean() * 100 if len(env_df) > 0 else 0
        line = f"║  {env.capitalize():12} {len(env_df):>4} models, {env_rate:>5.1f}%"
        report += f"\n{line:56}║"
    
    report += f"""
╠═══════════════════════════════════════════════════════╣
║  COMPLIANCE STATUS                                    ║
║  ─────────────────────────────────────────────────    ║
║  Bias Testing:    {bias_pct:>5.1f}%                              ║
║  Documentation:   {doc_pct:>5.1f}%                              ║
║  Security Review: {sec_pct:>5.1f}%                              ║
║  Avg Compliance:  {avg_compliance:>5.1f}%                              ║
╚═══════════════════════════════════════════════════════╝
"""
    return report


# Generate and display report
report = generate_governance_report(model_df)

In [ ]:
# Save DataFrame to CSV for external analysis
# Uncomment to save:
# model_df.to_csv("governance_data.csv", index=False)
# print("Data saved to governance_data.csv")

# Save report to file
# Uncomment to save:
# with open("governance_report.txt", "w") as f:
#     f.write(report)
# print("Report saved to governance_report.txt")


## 12. Cleanup

Close all figures to prevent memory leaks in Docker environments.

In [ ]:
# Close all matplotlib figures
plt.close("all")

---

## Summary

In this notebook, you learned to:

1. **Create governance dashboards** with multi-panel visualizations
2. **Analyze risk distributions** using violin plots, box plots, and KDE
3. **Track compliance status** and understand its impact on approvals
4. **Visualize audit trails** with time series analysis
5. **Build heatmaps** for multi-dimensional pattern analysis
6. **Generate governance reports** for stakeholder communication

### Visualization Best Practices

- Always set `MPLBACKEND=Agg` before importing matplotlib (required for Docker)
- Use `plt.close(fig)` after each visualization to prevent memory leaks
- Choose appropriate chart types for your data
- Use consistent color palettes (green=approved, red=denied)
- Add clear titles, labels, and legends

### Next Steps

1. **Connect to Real Data** - Replace simulated data with your governance logs
2. **Customize Dashboards** - Modify visualizations for your specific metrics
3. **Automate Reports** - Set up scheduled report generation
4. **Integrate with Monitoring** - Connect to Grafana or similar tools

### Feedback

Did this notebook help you understand governance visualization? We'd love your feedback!

- [Submit Feedback](../docs/feedback.md)
- [Report Issues](https://github.com/your-org/acgs2/issues)

---

*ACGS-2 Developer Onboarding - Governance Visualization Notebook*